In [4]:
import pandas as pd
import random
import numpy as np
import heapq
import math
from IPython.display import display, HTML

# Directions
Moves = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]

# Cost between moves
def move_cost(dx, dy):
    return math.hypot(dx, dy)

# Uniform Cost Search
def ucs(start, goal, maze):
    q = [(0, start, [])]
    visited = set()
    steps = 0
    while q:
        cost, (x, y), path = heapq.heappop(q)
        if (x, y) in visited:
            continue
        visited.add((x, y))
        steps += 1
        path = path + [(x, y)]
        if (x, y) == goal:
            return path, steps
        for dx, dy in Moves:
            nx, ny = x + dx, y + dy
            if 0 <= nx < 6 and 0 <= ny < 6 and maze[nx][ny] != 'X' and (nx, ny) not in visited:
                heapq.heappush(q, (cost + move_cost(dx, dy), (nx, ny), path))
    return None, steps

# A* Search
def astar(start, goal, maze):
    q = [(0, 0, start, [])]
    visited = set()
    steps = 0
    while q:
        _, cost, (x, y), path = heapq.heappop(q)
        if (x, y) in visited:
            continue
        visited.add((x, y))
        steps += 1
        path = path + [(x, y)]
        if (x, y) == goal:
            return path, steps
        for dx, dy in Moves:
            nx, ny = x + dx, y + dy
            if 0 <= nx < 6 and 0 <= ny < 6 and maze[nx][ny] != 'X' and (nx, ny) not in visited:
                g = cost + move_cost(dx, dy)
                h = max(abs(nx - goal[0]), abs(ny - goal[1]))
                heapq.heappush(q, (g + h, g, (nx, ny), path))
    return None, steps

# Style and display maze
def style_maze(df, path=None):
    df_copy = df.astype(str).copy()

    if path:
        for x, y in path:
            if df_copy.iat[x, y] not in ('S', 'G'):
                df_copy.iat[x, y] = '<span style="color:lime; font-weight:bold; font-size:20px;">*</span>'

    for i in range(6):
        for j in range(6):
            if df_copy.iat[i, j] == 'S':
                df_copy.iat[i, j] = '<span style="color:yellow; font-weight:bold; font-size:20px;">S</span>'
            elif df_copy.iat[i, j] == 'G':
                df_copy.iat[i, j] = '<span style="color:blue; font-weight:bold; font-size:20px;">G</span>'
            elif df_copy.iat[i, j] == 'X':
                df_copy.iat[i, j] = '<span style="color:red; font-weight:bold; font-size:20px;">X</span>'
            else:
                df_copy.iat[i, j] = f'<span style="font-size:20px;">{df_copy.iat[i, j]}</span>'

    return df_copy.to_html(
        escape=False,
        index=False,
        header=False,
        border=2,
        classes='custom-maze'
    )

# Final analysis
def analysis(name, steps, lengths):
    print(f"\n{name} Analysis:")
    print(f"Mean Steps: {np.mean(steps):.2f}")
    print(f"Var Steps: {np.var(steps):.2f}")
    print(f"Mean Path Length: {np.mean(lengths):.2f}")
    print(f"Var Path Length: {np.var(lengths):.2f}")

# ==== Main Experiment ====
ucs_steps, ucs_lengths = [], []
astar_steps, astar_lengths = [], []

html_rows = []

for run in range(3):
    # Create random maze
    maze = np.arange(36).reshape(6, 6).T
    df = pd.DataFrame(maze)

    start_num = random.choice(range(0, 11))
    goal_num = random.choice(range(24, 36))
    df.replace(start_num, 'S', inplace=True)
    df.replace(goal_num, 'G', inplace=True)
    barriers = random.sample(list(set(range(36)) - {start_num, goal_num}), 4)
    for b in barriers:
        df.replace(b, 'X', inplace=True)

    # Find start and goal
    start = goal = None
    for i in range(6):
        for j in range(6):
            if df.iat[i, j] == 'S':
                start = (i, j)
            if df.iat[i, j] == 'G':
                goal = (i, j)

    # UCS
    path_ucs, steps_ucs = ucs(start, goal, df.values)
    if path_ucs:
        ucs_steps.append(steps_ucs)
        ucs_lengths.append(len(path_ucs))
    else:
        ucs_steps.append(0)
        ucs_lengths.append(0)

    # A*
    path_astar, steps_astar = astar(start, goal, df.values)
    if path_astar:
        astar_steps.append(steps_astar)
        astar_lengths.append(len(path_astar))
    else:
        astar_steps.append(0)
        astar_lengths.append(0)

    # Print per-run results
    print(f"\nRun {run + 1}:")
    if path_ucs:
        print(f"  UCS -> Steps taken: {steps_ucs}, Path length: {len(path_ucs)}")
    else:
        print("  UCS -> No path found.")

    if path_astar:
        print(f"  A*  -> Steps taken: {steps_astar}, Path length: {len(path_astar)}")
    else:
        print("  A*  -> No path found.")

    # Prepare HTML for this run
    maze_html = style_maze(df)
    ucs_html = style_maze(df, path_ucs) if path_ucs else "<p style='color:red;'>No UCS Path</p>"
    astar_html = style_maze(df, path_astar) if path_astar else "<p style='color:red;'>No A* Path</p>"

    html_row = f"""
    <tr>
        <td style="vertical-align:top;">{maze_html}</td>
        <td style="vertical-align:top;">{ucs_html}</td>
        <td style="vertical-align:top;">{astar_html}</td>
    </tr>
    """
    html_rows.append(html_row)

# Display all runs together
final_html = f"""
<style>
    .custom-maze td {{
        width: 30px;
        height: 30px;
        text-align: center;
        vertical-align: middle;
    }}
</style>

<table style="border-collapse: separate; border-spacing: 100px 40px;">
    <thead>
        <tr>
            <th>Original Maze</th>
            <th>UCS Path</th>
            <th>A* Path</th>
        </tr>
    </thead>
    <tbody>
        {''.join(html_rows)}
    </tbody>
</table>
"""
display(HTML(final_html))

# Perform Final Analysis
analysis("UCS", ucs_steps, ucs_lengths)
analysis("A*", astar_steps, astar_lengths)



Run 1:
  UCS -> Steps taken: 29, Path length: 6
  A*  -> Steps taken: 13, Path length: 6

Run 2:
  UCS -> Steps taken: 26, Path length: 6
  A*  -> Steps taken: 10, Path length: 6

Run 3:
  UCS -> Steps taken: 19, Path length: 5
  A*  -> Steps taken: 5, Path length: 5



UCS Analysis:
Mean Steps: 24.67
Var Steps: 17.56
Mean Path Length: 5.67
Var Path Length: 0.22

A* Analysis:
Mean Steps: 9.33
Var Steps: 10.89
Mean Path Length: 5.67
Var Path Length: 0.22
